### Own layer

In [1]:
import keras
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

from keras import initializers 
igu = initializers.get('glorot_uniform')
iz = initializers.get('zeros')

Using TensorFlow backend.


In [2]:
class SFC(Layer):
    # FC: Simplified fully connected layer
    def __init__(self, No, **kwargs):
        self.No = No
        super().__init__(**kwargs)

    def build(self, inshape):
        self.w = self.add_weight("w", (inshape[1], self.No),
                                 initializer=igu)
        self.b = self.add_weight("b", (self.No,),
                                 initializer=iz)        
        super().build(inshape)  

    def call(self, x):
        return K.dot(x, self.w) + self.b

    def compute_output_shape(self, inshape):
        return (inshape[0], self.No)

In [3]:
def main():
    x = np.array([0, 1, 2, 3, 4]) 
    y = x * 2 + 1

    model = keras.models.Sequential()
    model.add(SFC(1, input_shape=(1,)))
    model.compile('SGD', 'mse')

    model.fit(x[:2], y[:2], epochs=1000, verbose=0)
    print('Targets:',y[2:])
    print('Predictions:', model.predict(x[2:]).flatten())


if __name__ == '__main__':
    main()

InternalError:  Blas GEMV launch failed:  m=1, n=2
	 [[node sfc_1/MatMul (defined at C:\Users\USER\anaconda3\envs\tensorflow_py37\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_252]

Function call stack:
keras_scratch_graph


### keras with tensorflow

In [1]:
import tensorflow as tf
tf.__version__
sess = tf.compat.v1.Session()

from keras import backend as K
tf.compat.v1.keras.backend.set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.metrics import categorical_accuracy, categorical_crossentropy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

class DNN():
    def __init__(self, Nin, Nh_l, Nout):
        self.X_ph = tf.placeholder(tf.float32, shape=(None, Nin))
        self.L_ph = tf.placeholder(tf.float32, shape=(None, Nout))
        
        # Modeling
        H = Dense(Nh_l[0], activation='relu')(self.X_ph)
        H = Dropout(0.5)(H)
        H = Dense(Nh_l[1], activation='relu')(H) 
        H = Dropout(0.25)(H)
        self.Y_tf = Dense(Nout, activation='softmax')(H)
        
        # Operation
        self.Loss_tf = tf.reduce_mean(
            categorical_crossentropy(self.L_ph, self.Y_tf))
        self.Train_tf = tf.train.AdamOptimizer().minimize(self.Loss_tf)
        self.Acc_tf = categorical_accuracy(self.L_ph, self.Y_tf)
        self.Init_tf = tf.global_variables_initializer()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import numpy as np
from keras import datasets  # mnist
from keras.utils import np_utils  # to_categorical

def Data_func():
    (X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

    Y_train = np_utils.to_categorical(y_train)
    Y_test = np_utils.to_categorical(y_test)

    L, W, H = X_train.shape
    X_train = X_train.reshape(-1, W * H)
    X_test = X_test.reshape(-1, W * H)

    X_train = X_train / 255.0
    X_test = X_test / 255.0

    return (X_train, Y_train), (X_test, Y_test)

In [4]:
from keraspp.skeras import plot_loss, plot_acc
import matplotlib.pyplot as plt

def run(model, data, sess, epochs, batch_size=100):
    # epochs = 2
    # batch_size = 100
    (X_train, Y_train), (X_test, Y_test) = data
    sess.run(model.Init_tf)
    with sess.as_default():
        N_tr = X_train.shape[0]
        for epoch in range(epochs):
            for b in range(N_tr // batch_size):
                X_tr_b = X_train[batch_size * (b-1):batch_size * b]
                Y_tr_b = Y_train[batch_size * (b-1):batch_size * b]

                model.Train_tf.run(feed_dict={model.X_ph: X_tr_b, model.L_ph: Y_tr_b, K.learning_phase(): 1})
            loss = sess.run(model.Loss_tf, feed_dict={model.X_ph: X_test, model.L_ph: Y_test, K.learning_phase(): 0})
            acc = model.Acc_tf.eval(feed_dict={model.X_ph: X_test, model.L_ph: Y_test, K.learning_phase(): 0})
            print("Epoch {0}: loss = {1:.3f}, acc = {2:.3f}".format(epoch, loss, np.mean(acc)))

In [5]:
def main():
    Nin = 784
    Nh_l = [100, 50]
    number_of_class = 10
    Nout = number_of_class

    data = Data_func()
    model = DNN(Nin, Nh_l, Nout)

    run(model, data, sess, 10, 100)


if __name__ == '__main__': 
    main()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 0: loss = 0.217, acc = 0.935
Epoch 1: loss = 0.168, acc = 0.949
Epoch 2: loss = 0.141, acc = 0.956
Epoch 3: loss = 0.133, acc = 0.960
Epoch 4: loss = 0.123, acc = 0.963
Epoch 5: loss = 0.115, acc = 0.965
Epoch 6: loss = 0.110, acc = 0.967
Epoch 7: loss = 0.107, acc = 0.968
Epoch 8: loss = 0.106, acc = 0.969
Epoch 9: loss = 0.101, acc = 0.970
